In [ ]:
# Importing libraries
import pyfabricops as pf
pf.set_auth_provider('env') 

# Setup logging
pf.setup_logging(format_style='minimal')


In [ ]:
# Reading SQL on premisses payloads
import json

connections_path = '../connections.json'

with open(connections_path, 'r', encoding='utf-8') as f:
    connections = json.load(f)

sql_on_premises_connections = connections['sql_on_premises_connections']

print(json.dumps(sql_on_premises_connections, indent=4))

In [ ]:
# Loading username and password for the connection:
from dotenv import load_dotenv
import os

load_dotenv()
username = os.getenv('DATABASE_USERNAME')
password = os.getenv('DATABASE_PASSWORD')

In [ ]:
for connection_key, connection_data in sql_on_premises_connections.items():
    
    # Retrieving connection details
    display_name = connection_data['display_name']
    gateway_id = connection_data['gateway_id']
    server = connection_data['server']
    database = connection_data['database']
    roles = connection_data['roles']
    

    # Creating SQL on premises connection
    print(f"Creating SQL on premises connection: {display_name}...") 
    connection_created = pf.create_sql_on_premises_connection(
        display_name=display_name,
        gateway_id=gateway_id,
        server = server,
        database = database,
        username=username,
        password=password
    )

    # Extracting connection ID
    connection_id = connection_created['id']
    print(f"Connection created with ID: {connection_id}")

    # Assigning roles to the connection
    for role in roles:
        # Assigning role to the connection
        print(f"Assigning role {role['role']} to user {role['user_uuid']} ({role['user_type']}) for connection {display_name}...")
        pf.add_connection_role_assignment(
            connection=connection_id,
            user_uuid=role['user_uuid'],
            user_type=role['user_type'],
            role=role['role'],
        )

    # Storing connection ID in the connections dictionary
    connections['sql_on_premises_connections'][display_name]['connection_id'] = connection_id
    print(f"Connection {display_name} created and roles assigned successfully.\n")

print(json.dumps(connections, indent=4))

In [ ]:
# Saving updated connections to the file
with open(connections_path, 'w', encoding='utf-8') as f:
    json.dump(connections, f, indent=4)